In [1]:
!pip install pandas requests beautifulsoup4 lxml

In [2]:
!pip install pandas requests


In [3]:
import pandas as pd

# Load your CSV from the data folder
file_path = "./data/mlb_betting_dashboard_template.csv"
dashboard = pd.read_csv(file_path)

dashboard.head()  # Show the empty template

,Game,SP (ERA/WHIP),Bullpen ERA,Team RPG (L7),Team OPS (L7),Opp RPG (L7),Opp OPS (L7),Weather,Notes


In [4]:
import requests
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo


# Define your teams of interest
teams_of_interest = ['NYM', 'LAD', 'NYY', 'DET', 'SEA', 'CHC']

# Get today's date in the required format
today = datetime.now(ZoneInfo("America/Los_Angeles")).strftime('%Y-%m-%d')

# MLB API endpoint for today's schedule
url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}&hydrate=team,linescore,probablePitcher'

# Fetch the schedule data
response = requests.get(url)
data = response.json()

# Prepare a list to store the data
games_list = []

# Iterate through the games
for date_info in data['dates']:
    for game in date_info['games']:
        home_team = game['teams']['home']['team']['abbreviation']
        away_team = game['teams']['away']['team']['abbreviation']
        home_pitcher = game['teams']['home'].get('probablePitcher', {}).get('fullName', 'TBD')
        away_pitcher = game['teams']['away'].get('probablePitcher', {}).get('fullName', 'TBD')

        # Check if either team is in your list
        if home_team in teams_of_interest or away_team in teams_of_interest:
            games_list.append({
                'Home Team': home_team,
                'Away Team': away_team,
                'Home Pitcher': home_pitcher,
                'Away Pitcher': away_pitcher
            })

# Convert the list to a DataFrame
df = pd.DataFrame(games_list)

# Display the DataFrame
df

,Home Team,Away Team,Home Pitcher,Away Pitcher
0,DET,BOS,Tyler Holton,Brayan Bello
1,NYM,PIT,Kodai Senga,Mitch Keller
2,CHC,MIA,Ben Brown,Valente Bellozo
3,SEA,NYY,Bryan Woo,Max Fried
4,LAD,ATH,Landon Knack,Jeffrey Springs


In [5]:
import requests
import pandas as pd
from datetime import datetime
from zoneinfo import ZoneInfo


# Step 1: Get today's date
today = datetime.now(ZoneInfo("America/Los_Angeles")).strftime('%Y-%m-%d')

# Step 2: Get today's games from the MLB API
schedule_url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}&hydrate=team,linescore,probablePitcher'
schedule_response = requests.get(schedule_url)
schedule_data = schedule_response.json()

# Step 3: Define your teams
teams_of_interest = ['NYM', 'LAD', 'NYY', 'DET', 'SEA', 'CHC']

# Step 4: Extract starting pitchers and team info (includes opponent pitchers)
pitchers = []

for date_info in schedule_data["dates"]:
    for game in date_info["games"]:
        home_team = game["teams"]["home"]["team"]["abbreviation"]
        away_team = game["teams"]["away"]["team"]["abbreviation"]

        home_pitcher = game["teams"]["home"].get("probablePitcher")
        away_pitcher = game["teams"]["away"].get("probablePitcher")

        # Add both pitchers regardless of team
        if home_pitcher:
            pitchers.append({
                "Team": home_team,
                "Pitcher Name": home_pitcher["fullName"],
                "Pitcher ID": home_pitcher["id"]
            })
        if away_pitcher:
            pitchers.append({
                "Team": away_team,
                "Pitcher Name": away_pitcher["fullName"],
                "Pitcher ID": away_pitcher["id"]
            })

# Step 5: For each pitcher, get last 3 starts and calculate ERA and WHIP
pitcher_stats = []

for p in pitchers:
    player_id = p['Pitcher ID']

    # NEW: Get handedness from player profile
    info_url = f"https://statsapi.mlb.com/api/v1/people/{player_id}"
    info_response = requests.get(info_url)
    info_data = info_response.json()
    handedness = info_data.get("people", [{}])[0].get("pitchHand", {}).get("code", "")

    logs_url = f"https://statsapi.mlb.com/api/v1/people/{player_id}/stats?stats=gameLog&group=pitching"
    logs_response = requests.get(logs_url)
    logs_data = logs_response.json()

    if 'stats' not in logs_data or not logs_data['stats']:
        continue

    game_logs = logs_data['stats'][0]['splits']
    starts = []

    for game in game_logs:
        stat = game['stat']
        if stat['gamesStarted'] > 0:
            starts.append({
                "ER": stat['earnedRuns'],
                "IP": stat['inningsPitched'],
                "BB": stat['baseOnBalls'],
                "H": stat['hits']
            })
        if len(starts) == 3:
            break

    if len(starts) == 0:
        continue
    elif len(starts) < 3:
        pitcher_stats.append({
            "Team": p["Team"],
            "Pitcher": p["Pitcher Name"],
            "ERA (Last 3 Starts)": "Fewer than 3 Starts",
            "WHIP (Last 3 Starts)": "Fewer than 3 Starts",
            "Pitcher Handedness": handedness.upper()
        })
        continue

    def parse_ip(ip_str):
        if '.' in ip_str:
            whole, partial = ip_str.split('.')
            return int(whole) + (1 if partial == '1' else 2)/3 if partial in ['1', '2'] else int(whole)
        return float(ip_str)

    total_er = sum(s['ER'] for s in starts)
    total_bb = sum(s['BB'] for s in starts)
    total_h = sum(s['H'] for s in starts)
    total_ip = sum(parse_ip(s['IP']) for s in starts)

    if total_ip == 0:
        continue

    era = round((total_er / total_ip) * 9, 2)
    whip = round((total_bb + total_h) / total_ip, 2)

    pitcher_stats.append({
        "Team": p["Team"],
        "Pitcher": p["Pitcher Name"],
        "ERA (Last 3 Starts)": era,
        "WHIP (Last 3 Starts)": whip,
        "Pitcher Handedness": handedness.upper()
    })



# Step 6: Display the result
pitcher_stats_df = pd.DataFrame(pitcher_stats)
pitcher_stats_df

,Team,Pitcher,ERA (Last 3 Starts),WHIP (Last 3 Starts),Pitcher Handedness
0,CLE,Logan Allen,2.3,1.4,L
1,MIL,Quinn Priester,1.93,1.43,R
2,DET,Tyler Holton,Fewer than 3 Starts,Fewer than 3 Starts,L
3,BOS,Brayan Bello,2.55,1.19,R
4,CIN,Andrew Abbott,3.6,1.13,L
5,CWS,Jonathan Cannon,5.79,1.64,R
6,TOR,José Berríos,4.58,1.42,R
7,TB,Shane Baz,1.42,0.84,R
8,NYM,Kodai Senga,1.06,1.0,R
9,PIT,Mitch Keller,4.24,1.35,R


In [6]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo


# Get today's date
today = datetime.now(ZoneInfo("America/Los_Angeles")).strftime('%Y-%m-%d')

# Get today's games from MLB API
schedule_url = f'https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}&hydrate=team,linescore,probablePitcher'
schedule_response = requests.get(schedule_url)
schedule_data = schedule_response.json()

# Step 3: Build full matchups (fixes missing opponent data)
teams_of_interest = ['NYM', 'LAD', 'NYY', 'DET', 'SEA', 'CHC']
matchups = []

for date_info in schedule_data['dates']:
    for game in date_info['games']:
        home_team = game['teams']['home']['team']
        away_team = game['teams']['away']['team']
        home_abbr = home_team['abbreviation']
        away_abbr = away_team['abbreviation']

        # Include the game if either team is one of your 6
        if home_abbr in teams_of_interest or away_abbr in teams_of_interest:
            if home_abbr in teams_of_interest:
                my_team = home_abbr
                my_team_id = home_team['id']
                opponent = away_abbr
                opponent_id = away_team['id']
            else:
                my_team = away_abbr
                my_team_id = away_team['id']
                opponent = home_abbr
                opponent_id = home_team['id']

            matchups.append({
                "Team": my_team,
                "Opponent": opponent,
                "Team ID": my_team_id,
                "Opponent ID": opponent_id
            })


# Function to calculate RPG and OPS for last 7 games
def get_rpg_ops(team_id):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=14)
    start = start_date.strftime('%Y-%m-%d')
    end = end_date.strftime('%Y-%m-%d')

    url = f"https://statsapi.mlb.com/api/v1/teams/{team_id}/stats?stats=gameLog&startDate={start}&endDate={end}&group=hitting"
    response = requests.get(url)
    data = response.json()

    if not data.get("stats") or not data["stats"][0].get("splits"):
        return None, None

    splits = data["stats"][0]["splits"][:7]  # Last 7 games
    total_runs = sum(int(game["stat"].get("runs", 0)) for game in splits)
    total_ops = sum(float(game["stat"].get("ops", 0)) for game in splits)

    rpg = round(total_runs / len(splits), 2)
    ops = round(total_ops / len(splits), 3)
    return rpg, ops

# Collect stats
team_stats = []

for matchup in matchups:
    team_rpg, team_ops = get_rpg_ops(matchup["Team ID"])
    opp_rpg, opp_ops = get_rpg_ops(matchup["Opponent ID"])

    team_stats.append({
        "Team": matchup["Team"],
        "Opponent": matchup["Opponent"],
        "Team RPG (L7)": team_rpg,
        "Team OPS (L7)": team_ops,
        "Opp RPG (L7)": opp_rpg,
        "Opp OPS (L7)": opp_ops
    })

# Display results
rpg_ops_df = pd.DataFrame(team_stats)
rpg_ops_df

,Team,Opponent,Team RPG (L7),Team OPS (L7),Opp RPG (L7),Opp OPS (L7)
0,DET,BOS,7.57,0.866,4.57,0.740
1,NYM,PIT,5.14,0.853,2.14,0.624
2,CHC,MIA,6.43,0.819,4.29,0.662
3,SEA,NYY,5.86,0.755,6.29,0.801
4,LAD,ATH,7.57,0.899,4.86,0.750


In [7]:
# Filter pitcher_stats_df to only include your tracked teams
your_teams = ['NYM', 'LAD', 'NYY', 'DET', 'SEA', 'CHC']
filtered_pitchers = pitcher_stats_df[pitcher_stats_df['Team'].isin(your_teams)]

filtered_pitchers.head()

,Team,Pitcher,ERA (Last 3 Starts),WHIP (Last 3 Starts),Pitcher Handedness
2,DET,Tyler Holton,Fewer than 3 Starts,Fewer than 3 Starts,L
8,NYM,Kodai Senga,1.06,1.0,R
12,CHC,Ben Brown,4.8,1.6,R
20,SEA,Bryan Woo,2.84,0.95,R
21,NYY,Max Fried,1.56,1.21,L


In [8]:
# Step 1: Drop duplicates in filtered_pitchers to ensure one pitcher per team
filtered_pitchers = filtered_pitchers.drop_duplicates(subset=["Team"])

# Step 1: Merge your team’s pitcher stats with RPG/OPS
dashboard = pd.merge(filtered_pitchers, rpg_ops_df.drop_duplicates(subset=["Team", "Opponent"]), on="Team", how="left")

# Step 1b: Add Pitcher Handedness from pitcher_stats_df (deduped)
pitcher_handedness = pitcher_stats_df.drop_duplicates(subset=["Team"])[["Team", "Pitcher Handedness"]]
dashboard = pd.merge(dashboard, pitcher_handedness, on="Team", how="left")

# Step 2: Merge opponent pitcher stats (deduped and renamed to avoid column name conflicts)
opponent_pitcher_stats = pitcher_stats_df.rename(columns={
    "Team": "Opponent",
    "Pitcher": "Opponent Pitcher",
    "ERA (Last 3 Starts)": "Opponent ERA (Last 3 Starts)",
    "WHIP (Last 3 Starts)": "Opponent WHIP (Last 3 Starts)"
}).drop_duplicates(subset=["Opponent"])
dashboard = pd.merge(dashboard, opponent_pitcher_stats, on="Opponent", how="left")

# Step 2b: Add Opponent Handedness (deduped)
opponent_handedness = pitcher_stats_df.rename(columns={
    "Team": "Opponent",
    "Pitcher Handedness": "Opponent Handedness"
})[["Opponent", "Opponent Handedness"]].drop_duplicates(subset=["Opponent"])
dashboard = pd.merge(dashboard, opponent_handedness, on="Opponent", how="left")

# Step 2c: Map Opponent Handedness Risk
dashboard["Opponent Handedness Risk"] = dashboard["Opponent Handedness"].map({
    "R": "rhh",
    "L": "lhh"
})

# Step 3: Reorder columns
dashboard = dashboard[[
    "Team", "Opponent",
    "Pitcher", "ERA (Last 3 Starts)", "WHIP (Last 3 Starts)",
    "Opponent Pitcher", "Opponent ERA (Last 3 Starts)", "Opponent WHIP (Last 3 Starts)",
    "Team RPG (L7)", "Team OPS (L7)",
    "Opp RPG (L7)", "Opp OPS (L7)",
    "Pitcher Handedness", "Opponent Handedness", "Opponent Handedness Risk"
]]

# Step 4: Final result
final_dashboard = dashboard

# Step 5: True matchup-based lean logic
def generate_lean(row):
    try:
        era = float(row["ERA (Last 3 Starts)"])
        whip = float(row["WHIP (Last 3 Starts)"])
        opp_era = float(row["Opponent ERA (Last 3 Starts)"])
        opp_whip = float(row["Opponent WHIP (Last 3 Starts)"])
        ops = float(row["Team OPS (L7)"])
        opp_ops = float(row["Opp OPS (L7)"])
        rpg = float(row["Team RPG (L7)"])
        opp_rpg = float(row["Opp RPG (L7)"])
    except:
        return "Stay Away"

    # Pitching edge thresholds
    pitching_edge = (opp_era - era >= 1.0) and (opp_whip - whip >= 0.15)
    pitching_disadv = (era - opp_era >= 1.0) and (whip - opp_whip >= 0.15)

    # Offensive edge thresholds
    batting_edge = (ops - opp_ops >= 0.05) and (rpg - opp_rpg >= 0.5)
    batting_disadv = (opp_ops - ops >= 0.05) and (opp_rpg - rpg >= 0.5)

    # Decision logic
    if pitching_edge and batting_edge:
        return "Lean RL"
    elif pitching_edge or batting_edge:
        return "Lean ML"
    elif pitching_disadv and batting_disadv:
        return "Fade ML"
    else:
        return "Stay Away"

# Step 6: Strength classification based on your pitcher
def classify_strength(row):
    try:
        era = float(row["ERA (Last 3 Starts)"])
        whip = float(row["WHIP (Last 3 Starts)"])
    except:
        return "Normal"

    if row["Lean Call"].startswith("Lean ML"):
        if era < 2.0 and whip < 1.00:
            return "Strong"
        elif era < 3.0:
            return "Moderate"
    elif row["Lean Call"].startswith("Lean RL"):
        if era < 2.5 and whip < 1.05:
            return "Strong"
        elif era < 3.5:
            return "Moderate"
    elif row["Lean Call"].startswith("Fade ML"):
        if era > 5.0 or whip > 1.45:
            return "Strong"
        elif era > 4.5:
            return "Moderate"
    return "Normal"

# Step 7: Add Lean + Strength
final_dashboard["Lean Call"] = final_dashboard.apply(generate_lean, axis=1)
final_dashboard["Lean Strength"] = final_dashboard.apply(classify_strength, axis=1)
final_dashboard["Lean Call"] = final_dashboard["Lean Call"] + " (" + final_dashboard["Lean Strength"] + ")"
final_dashboard = final_dashboard.drop(columns=["Lean Strength"])

# Step 8: Add Lean Reason column
def explain_lean_reason(row):
    try:
        era_diff = float(row["Opponent ERA (Last 3 Starts)"]) - float(row["ERA (Last 3 Starts)"])
        whip_diff = float(row["Opponent WHIP (Last 3 Starts)"]) - float(row["WHIP (Last 3 Starts)"])
        ops_diff = float(row["Team OPS (L7)"]) - float(row["Opp OPS (L7)"])
        rpg_diff = float(row["Team RPG (L7)"]) - float(row["Opp RPG (L7)"])
        return f"ERA {era_diff:+.2f}, WHIP {whip_diff:+.2f}, OPS {ops_diff:+.3f}, RPG {rpg_diff:+.2f}"
    except:
        return ""

final_dashboard["Lean Reason"] = final_dashboard.apply(explain_lean_reason, axis=1)

In [9]:
final_dashboard.head()

,Team,Opponent,Pitcher,ERA (Last 3 Starts),WHIP (Last 3 Starts),Opponent Pitcher,Opponent ERA (Last 3 Starts),Opponent WHIP (Last 3 Starts),Team RPG (L7),Team OPS (L7),Opp RPG (L7),Opp OPS (L7),Pitcher Handedness,Opponent Handedness,Opponent Handedness Risk,Lean Call,Lean Reason
0,DET,BOS,Tyler Holton,Fewer than 3 Starts,Fewer than 3 Starts,Brayan Bello,2.55,1.19,7.57,0.866,4.57,0.740,R,R,rhh,Stay Away (Normal),
1,NYM,PIT,Kodai Senga,1.06,1.0,Mitch Keller,4.24,1.35,5.14,0.853,2.14,0.624,R,R,rhh,Lean RL (Strong),"ERA +3.18, WHIP +0.35, OPS +0.229, RPG +3.00"
2,CHC,MIA,Ben Brown,4.8,1.6,Valente Bellozo,4.97,1.66,6.43,0.819,4.29,0.662,R,R,rhh,Lean ML (Normal),"ERA +0.17, WHIP +0.06, OPS +0.157, RPG +2.14"
3,SEA,NYY,Bryan Woo,2.84,0.95,Max Fried,1.56,1.21,5.86,0.755,6.29,0.801,L,L,lhh,Stay Away (Normal),"ERA -1.28, WHIP +0.26, OPS -0.046, RPG -0.43"
4,NYY,NaN,Max Fried,1.56,1.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stay Away (Normal),"ERA +nan, WHIP +nan, OPS +nan, RPG +nan"


In [10]:
filtered_pitchers.head()

,Team,Pitcher,ERA (Last 3 Starts),WHIP (Last 3 Starts),Pitcher Handedness
2,DET,Tyler Holton,Fewer than 3 Starts,Fewer than 3 Starts,L
8,NYM,Kodai Senga,1.06,1.0,R
12,CHC,Ben Brown,4.8,1.6,R
20,SEA,Bryan Woo,2.84,0.95,R
21,NYY,Max Fried,1.56,1.21,L


In [12]:
# Final display/export structure
final_dashboard = final_dashboard[[
    "Team", "Opponent", "Pitcher", "Pitcher Handedness",
    "ERA (Last 3 Starts)", "WHIP (Last 3 Starts)",
    "Opponent Pitcher", "Opponent Handedness",
    "Opponent ERA (Last 3 Starts)", "Opponent WHIP (Last 3 Starts)",
    "Team RPG (L7)", "Team OPS (L7)",
    "Opp RPG (L7)", "Opp OPS (L7)",
    "Lean Call", "Lean Reason"
]]

# Save updated dashboard
final_dashboard.to_csv(f"data/mlb_dashboard_{today}.csv", index=False)
final_dashboard.head()

,Team,Opponent,Pitcher,Pitcher Handedness,ERA (Last 3 Starts),WHIP (Last 3 Starts),Opponent Pitcher,Opponent Handedness,Opponent ERA (Last 3 Starts),Opponent WHIP (Last 3 Starts),Team RPG (L7),Team OPS (L7),Opp RPG (L7),Opp OPS (L7),Lean Call,Lean Reason
0,DET,BOS,Tyler Holton,R,Fewer than 3 Starts,Fewer than 3 Starts,Brayan Bello,R,2.55,1.19,7.57,0.866,4.57,0.740,Stay Away (Normal),
1,NYM,PIT,Kodai Senga,R,1.06,1.0,Mitch Keller,R,4.24,1.35,5.14,0.853,2.14,0.624,Lean RL (Strong),"ERA +3.18, WHIP +0.35, OPS +0.229, RPG +3.00"
2,CHC,MIA,Ben Brown,R,4.8,1.6,Valente Bellozo,R,4.97,1.66,6.43,0.819,4.29,0.662,Lean ML (Normal),"ERA +0.17, WHIP +0.06, OPS +0.157, RPG +2.14"
3,SEA,NYY,Bryan Woo,L,2.84,0.95,Max Fried,L,1.56,1.21,5.86,0.755,6.29,0.801,Stay Away (Normal),"ERA -1.28, WHIP +0.26, OPS -0.046, RPG -0.43"
4,NYY,NaN,Max Fried,NaN,1.56,1.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stay Away (Normal),"ERA +nan, WHIP +nan, OPS +nan, RPG +nan"


In [13]:
# Append daily results to bet history log
from pathlib import Path

log_file = Path("data/bet_history.csv")
final_dashboard["Date"] = today

columns_to_log = [
    "Date", "Team", "Opponent", "Pitcher", "Opponent Pitcher",
    "ERA (Last 3 Starts)", "WHIP (Last 3 Starts)",
    "Team OPS (L7)", "Opp OPS (L7)",
    "Lean Call"
]

if log_file.exists():
    history_df = pd.read_csv(log_file)
    updated_df = pd.concat([history_df, final_dashboard[columns_to_log]], ignore_index=True)
else:
    updated_df = final_dashboard[columns_to_log]

updated_df.to_csv(log_file, index=False)

cd C:\Users\baile\Projects\mlb-dashboard

streamlit run dashboard_app.py

If not updating: streamlit run dashboard_app.py --clear-cache